In [1]:
from terminusdb_client import WOQLQuery as WQ
from terminusdb_client import WOQLDataFrame as WDF
from terminusdb_client import WOQLClient, WOQLView

server_url = "https://127.0.0.1:6363"
user = "admin"
account = "admin"
key = "root"
dbpedia = "dbpedia"
dbid = "seshat_combined"
repository = "local"

client = WOQLClient(server_url)
connection = client.connect(user=user,account=account,key=key,db=dbpedia)

In [11]:
conflicts = WQ().woql_and(
    WQ().triple("v:Battle", "type", "scm:MilitaryConflict"),
    WQ().triple("v:Battle", "http://xmlns.com/foaf/0.1/name", "v:Battle_Name"),
    WQ().triple("v:Battle", "scm:combatant", "v:Combatant"),
    WQ().triple("v:Battle", "scm:date", "v:Date"),
    WQ().re("(....)-..-..", "v:Date", ["v:All", "v:Year"]),
    WQ().cast("v:Year", "xsd:integer", "v:Year_Integer")
)

date_query = WQ().select("v:Polity", "v:Start_Integer", "v:End_Integer").woql_and(
    WQ().triple("v:Polity", "type", WQ().iri("terminusdb:///schema#Polity")),
    WQ().triple("v:Polity", "terminusdb:///schema#start", "v:Start_Range"),
    WQ().cast("v:Start_Range", "xsd:string", "v:Start_String"),
    WQ().cast("v:Start_String", "xsd:integer", "v:Start_Integer"),
    WQ().greater("v:End", "v:Start_Integer"),
    WQ().triple("v:Polity", "terminusdb:///schema#end", "v:End_Range"),
    WQ().cast("v:End_Range", "xsd:string", "v:End_String"),
    WQ().cast("v:End_String", "xsd:integer", "v:End_Integer"),
    WQ().greater("v:End_Integer", "v:Begin")
)

name_match = WQ().woql_and(
    WQ().triple("v:Polity", "label", "v:Name"),
    WQ().triple("v:Polity", "terminusdb:///schema#alternative_names", "v:YYY"),
    WQ().triple("v:YYY", "terminusdb:///schema#string", "v:AName"),
    WQ().re("(px)?([a-zA-Z\\- ]+)", "v:Combatant", ["v:Combatant_All", "v:Garbage", "v:Pattern"]),
    WQ().woql_or(
      WQ().like("v:Pattern", "v:Name", "v:Match"),
      WQ().like("v:Pattern", "v:AName", "v:Match")
    ), 
    WQ().greater("v:Match", 0.8)
)

candidate_polities = WQ().woql_and(
    conflicts,
    WQ().using(f"{user}/{dbid}/local/branch/conflict").woql_and(
        WQ().triple("v:Polity", "type", WQ().iri("terminusdb:///schema#Polity")),
        WQ().eq("v:Begin", "v:Year_Integer"),
        WQ().eq("v:End", "v:Year_Integer"),
        date_query, 
        name_match
    )
)

id_convert = WQ().woql_and(
    WQ().cast("v:Battle","xsd:string","v:Battle_String"),
    WQ().re("http://dbpedia.org/resource/(.*)", "v:Battle_String", ["v:All", "v:Doc_Name"]),
    WQ().concat(["terminusdb:///data/", "v:Doc_Name"], "v:ID_String"), 
    WQ().cast("v:ID_String", "owl:Thing", "v:Conflict_ID")
)

year_convert = WQ().woql_and(
    WQ().cast("v:Year_Integer", "xdd:integerRange", "v:Year_Range")
)

insertion = WQ().using(f"{user}/{dbid}/local/branch/conflict").woql_and(
    WQ().add_triple("v:Conflict_ID", "type", "terminusdb:///schema#Battle"),
    WQ().add_triple("v:Conflict_ID", "label", "v:Battle_Name"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#end", "v:Year_Range"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#start",  "v:Year_Range"),
    WQ().add_triple("v:Conflict_ID", "terminusdb:///schema#belligerent", "v:Polity")
)

id_convert.execute(client)

seshat_results = WQ().woql_and(candidate_polities, id_convert, year_convert, insertion).execute(client)

/usr/lib/python3.8/socket.py:740: ResourceWarning: unclosed <socket.socket fd=42, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 33950), raddr=('127.0.0.1', 6363)>
  self._sock = None


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [5]:
WDF.result_to_df(seshat_results)

,AName,All,Battle,Battle_Name,Combatant,Combatant_All,Date,End_Integer,Garbage,Match,Name,Pattern,Polity,Start_Integer,YYY,Year,Year_Integer
0,Last Afghan Empire,1757-11-11,http://dbpedia.org/resource/Battle_of_Amritsar...,Battle of Amritsar,Durrani Empire,Durrani Empire,1757-11-11,1826,,1.000000,Durrani Empire,Durrani Empire,terminusdb:///data/afdurrn,1747,terminusdb:///data/f0f17c9c812ef90f08eaa659900...,1757,1757
1,Sadozai Kingdom,1757-11-11,http://dbpedia.org/resource/Battle_of_Amritsar...,Battle of Amritsar,Durrani Empire,Durrani Empire,1757-11-11,1826,,1.000000,Durrani Empire,Durrani Empire,terminusdb:///data/afdurrn,1747,terminusdb:///data/f0f17c9c812ef90f08eaa659900...,1757,1757
2,Last Afghan Empire,1813-07-13,http://dbpedia.org/resource/Battle_of_Attock,Battle of Attock,24pxDurrani Empire,pxDurrani Empire,1813-07-13,1826,px,1.000000,Durrani Empire,Durrani Empire,terminusdb:///data/afdurrn,1747,terminusdb:///data/f0f17c9c812ef90f08eaa659900...,1813,1813
3,Sadozai Kingdom,1813-07-13,http://dbpedia.org/resource/Battle_of_Attock,Battle of Attock,24pxDurrani Empire,pxDurrani Empire,1813-07-13,1826,px,1.000000,Durrani Empire,Durrani Empire,terminusdb:///data/afdurrn,1747,terminusdb:///data/f0f17c9c812ef90f08eaa659900...,1813,1813
4,Last Afghan Empire,1758-04-28,http://dbpedia.org/resource/Battle_of_Attock_(...,Battle of Attock,Durrani Empire,Durrani Empire,1758-04-28,1826,,1.000000,Durrani Empire,Durrani Empire,terminusdb:///data/afdurrn,1747,terminusdb:///data/f0f17c9c812ef90f08eaa659900...,1758,1758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,Timurid Dynasty,1391-06-18,http://dbpedia.org/resource/Battle_of_the_Kond...,Battle of the Kondurcha River,25pxTimurid Empire,pxTimurid Empire,1391-06-18,1526,px,1.000000,Timurid Empire,Timurid Empire,terminusdb:///data/uztimur,1370,terminusdb:///data/8018ebe05a5163f4b2d144be7f0...,1391,1391
2334,Timurid Dynasty,1395-04-15,http://dbpedia.org/resource/Battle_of_the_Tere...,Battle of the Terek River,25pxTimurid Empire,pxTimurid Empire,1395-04-15,1526,px,1.000000,Timurid Empire,Timurid Empire,terminusdb:///data/uztimur,1370,terminusdb:///data/8018ebe05a5163f4b2d144be7f0...,1395,1395
2335,Timurid Dynasty,1403-08-22,http://dbpedia.org/resource/Siege_of_Birtvisi_...,Siege of Birtvisi,Timurid Empire,Timurid Empire,1403-08-22,1526,,1.000000,Timurid Empire,Timurid Empire,terminusdb:///data/uztimur,1370,terminusdb:///data/8018ebe05a5163f4b2d144be7f0...,1403,1403
2336,Ziyadid Dynasty,0900-10-03,http://dbpedia.org/resource/Battle_of_Gorgan_(...,Battle of Gorgan,Zaydid dynasty,Zaydid dynasty,0900-10-03,1037,,0.841080,Banu Ziyad,Zaydid dynasty,terminusdb:///data/yeziyad,822,terminusdb:///data/2ceba475ef505dc41e5ae1405ba...,0900,900
